<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/etl_vl_01raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INICIALIZACIÓN

In [ ]:
# @title INSTALACIÓN DE LIBRERÍAS
!pip install boto3
!pip install google-cloud-secret-manager
!pip install dateparser



In [ ]:
# @title IMPORTACIÓN DE LIBRERÍAS GCP

import boto3
from google.cloud import storage
import os

from google.cloud import bigquery  # Importa el cliente de BigQuery
from google.cloud import storage   # Importa el cliente de Cloud Storage
import pandas as pd
import io
import re
import unicodedata
import chardet  # Biblioteca para detectar codificaciones de texto

from google.colab import auth
auth.authenticate_user()

from google.cloud import secretmanager
import os

# Configura el cliente de Secret Manager
client = secretmanager.SecretManagerServiceClient()
project_id = "animum-dev-datawarehouse"

# Función para obtener un secreto por su nombre
def get_secret(secret_id):
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

# Obtén los valores de los secretos
aws_access_key_id = get_secret("AWS_ACCESS_KEY_ID_velneo")
aws_secret_access_key = get_secret("AWS_SECRET_ACCESS_KEY_velneo")

# Configura las variables de entorno
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-1'

print("Secretos configurados correctamente.")



Secretos configurados correctamente.


In [ ]:
# @title S3_folder_and_files_list()
def S3_folder_and_files_list(params: dict) -> dict:
    """
    Lista todos los archivos y subcarpetas a partir de una carpeta específica en un bucket de s3.

    Args:
        params (dict): Diccionario con las claves:
            - S3_bucket_name (str): Nombre del bucket de s3.
            - S3_folder_path (str): Ruta de la carpeta en el bucket de s3.

    Returns:
        dict: Diccionario con la estructura de carpetas y archivos:
            {
                'folders': {
                    'subcarpeta1/': {
                        'files': [lista de archivos en subcarpeta1],
                        'folders': {estructura recursiva de subcarpetas}
                    }
                },
                'files': [lista de archivos en la carpeta actual]
            }

    Raises:
        ValueError: Si falta algún parámetro en params.
        Exception: Si ocurre un error al listar los objetos.
    """
    import boto3
    import logging

    # Configurar el registro
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Validar parámetros
    S3_bucket_name = params.get('S3_bucket_name')
    S3_folder_path = params.get('S3_folder_path')

    if not S3_bucket_name or not S3_folder_path:
        raise ValueError("Faltan parámetros requeridos: 'S3_bucket_name' o 'S3_folder_path'.")

    try:
        # Cliente de s3
        s3_client = boto3.client('s3')

        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=S3_bucket_name, Prefix=S3_folder_path)

        structure = {'folders': {}, 'files': []}

        # Procesar los objetos obtenidos
        for page in pages:
            for obj in page.get('Contents', []):
                key = obj['Key']
                if key.endswith('/'):
                    # Es una carpeta
                    relative_path = key[len(S3_folder_path):]
                    if '/' not in relative_path.strip('/'):
                        structure['folders'][key] = {
                            'files': [],
                            'folders': {}
                        }
                else:
                    # Es un archivo
                    relative_path = key[len(S3_folder_path):]
                    if '/' not in relative_path:
                        structure['files'].append(key)

        return structure

    except Exception as e:
        logger.error(f"Error al listar objetos en s3: {e}")
        raise

In [ ]:
# @title S3_to_GCS_transfer_file()
def S3_to_GCS_transfer_file(config: dict):
    """
    Transfiere archivos desde un bucket S3 a un bucket de Google Cloud Storage.
    Muestra mensajes por pantalla en tiempo real.

    Args:
        config (dict): Diccionario con las claves:
            - S3_bucket_name (str): Nombre del bucket en S3.
            - S3_folder_path (str): Ruta de la carpeta en el bucket de S3.
            - GCS_bucket_name (str): Nombre del bucket en Google Cloud Storage.
            - filter_file (dict): Diccionario con opciones de filtro para los archivos:
                - use_bool (bool): Activar o desactivar los filtros.
                - name_include_patterns_list (list[str]): Lista de patrones a incluir por nombre.
                - name_exclude_patterns_list (list[str]): Lista de patrones a excluir por nombre.
                - extension_include_patterns_list (list[str]): Extensiones permitidas.
                - extension_exclude_patterns_list (list[str]): Extensiones excluidas.
                - min_size_kb (int): Tamaño mínimo del archivo en KB.
                - max_size_kb (int): Tamaño máximo del archivo en KB.
                - modified_after_date (str): Fecha mínima de modificación (YYYY-MM-DD).
                - modified_before_date (str): Fecha máxima de modificación (YYYY-MM-DD).
                - include_subfolders_bool (bool): Incluir subcarpetas en el filtro.
    """
    import boto3
    from google.cloud import storage
    import os
    from datetime import datetime

    start_time = datetime.now()
    print(f"\n=== Iniciando proceso de transferencia de archivos | {start_time} ===\n", flush=True)

    # Extraer parámetros
    S3_bucket_name = config['S3_bucket_name']
    S3_folder_path = config['S3_folder_path']
    GCS_bucket_name = config['GCS_bucket_name']
    filter_file = config.get('filter_file', {})

    print("Autenticando cliente de s3...", flush=True)
    s3_client = boto3.client('s3')

    print("Autenticando cliente de Google Cloud Storage...", flush=True)
    storage_client = storage.Client()
    bucket = storage_client.bucket(GCS_bucket_name)

    # Listar objetos en la carpeta s3
    print(f"Listando archivos en s3 bucket: '{S3_bucket_name}', carpeta: '{S3_folder_path}'\n", flush=True)
    s3_objects = s3_client.list_objects_v2(Bucket=S3_bucket_name, Prefix=S3_folder_path)

    if 'Contents' not in s3_objects:
        print("No se encontraron objetos en la carpeta s3 especificada.\n", flush=True)
        return

    # Extraer la lista de archivos (claves)
    s3_files = []
    for obj in s3_objects['Contents']:
        key = obj['Key']
        # Subcarpetas
        if not filter_file.get('include_subfolders_bool', True):
            relative_path = key[len(S3_folder_path):]
            if '/' in relative_path:
                continue
        s3_files.append(key)

    # Aplicar filtros si están habilitados
    if filter_file.get('use_bool', False):
        print("Aplicando filtros definidos en 'filter_file':", flush=True)
        print(filter_file, flush=True)

        # Filtrar por patrones en el nombre
        include_patterns = filter_file.get('name_include_patterns_list', [])
        exclude_patterns = filter_file.get('name_exclude_patterns_list', [])

        if include_patterns:
            before_count = len(s3_files)
            s3_files = [
                file for file in s3_files
                if any(pattern in os.path.basename(file) for pattern in include_patterns)
            ]
            after_count = len(s3_files)
            print(f"Filtrando por include_patterns={include_patterns}: {before_count} -> {after_count}", flush=True)

        if exclude_patterns:
            before_count = len(s3_files)
            s3_files = [
                file for file in s3_files
                if not any(pattern in os.path.basename(file) for pattern in exclude_patterns)
            ]
            after_count = len(s3_files)
            print(f"Filtrando por exclude_patterns={exclude_patterns}: {before_count} -> {after_count}", flush=True)

        # Filtrar por extensión
        include_extensions = filter_file.get('extension_include_patterns_list', [])
        exclude_extensions = filter_file.get('extension_exclude_patterns_list', [])

        if include_extensions:
            before_count = len(s3_files)
            s3_files = [file for file in s3_files if any(file.endswith(ext) for ext in include_extensions)]
            after_count = len(s3_files)
            print(f"Filtrando extensiones permitidas={include_extensions}: {before_count} -> {after_count}", flush=True)

        if exclude_extensions:
            before_count = len(s3_files)
            s3_files = [file for file in s3_files if not any(file.endswith(ext) for ext in exclude_extensions)]
            after_count = len(s3_files)
            print(f"Excluyendo extensiones={exclude_extensions}: {before_count} -> {after_count}", flush=True)

        # Filtrar por tamaño
        min_size_kb = filter_file.get('min_size_kb')
        max_size_kb = filter_file.get('max_size_kb')
        if min_size_kb or max_size_kb:
            print(f"Filtrando por tamaño: min_size_kb={min_size_kb}, max_size_kb={max_size_kb}", flush=True)
            before_count = len(s3_files)
            all_objects = {obj['Key']: obj for obj in s3_objects['Contents']}
            filtered_files = []
            for file in s3_files:
                obj = all_objects.get(file)
                if not obj:
                    continue
                file_size_kb = obj['Size'] / 1024
                if (min_size_kb and file_size_kb < min_size_kb):
                    continue
                if (max_size_kb and file_size_kb > max_size_kb):
                    continue
                filtered_files.append(file)
            s3_files = filtered_files
            after_count = len(s3_files)
            print(f"Filtrando por tamaño: {before_count} -> {after_count}", flush=True)

        # Filtrar por fecha de modificación
        modified_after = filter_file.get('modified_after_date')
        modified_before = filter_file.get('modified_before_date')
        if modified_after or modified_before:
            print(f"Filtrando por fecha. after={modified_after}, before={modified_before}", flush=True)
            before_count = len(s3_files)
            from datetime import datetime
            modified_after_dt = datetime.strptime(modified_after, "%Y-%m-%d") if modified_after else None
            modified_before_dt = datetime.strptime(modified_before, "%Y-%m-%d") if modified_before else None

            all_objects = {obj['Key']: obj for obj in s3_objects['Contents']}
            filtered_files = []
            for file in s3_files:
                obj = all_objects.get(file)
                if not obj:
                    continue
                last_modified = obj['LastModified']

                if modified_after_dt and last_modified < modified_after_dt:
                    continue
                if modified_before_dt and last_modified > modified_before_dt:
                    continue
                filtered_files.append(file)

            s3_files = filtered_files
            after_count = len(s3_files)
            print(f"Filtrado por fecha: {before_count} -> {after_count}", flush=True)

    # Verificar si hay archivos por transferir
    if not s3_files:
        print("No se encontraron archivos para transferir tras aplicar filtros.\n", flush=True)
        end_time = datetime.now()
        print(f"=== Proceso finalizado sin transferencias | {end_time} ===\n", flush=True)
        return

    print("=== Comenzando a transferir archivos ===\n", flush=True)
    transfer_log = []

    for s3_file in s3_files:
        temp_file_name = os.path.basename(s3_file)
        temp_file_path = f"/tmp/{temp_file_name}"

        print(f"Iniciando transferencia de '{s3_file}'", flush=True)
        try:
            # Descargar archivo desde s3
            print(f"Descargando: s3://{S3_bucket_name}/{s3_file} -> {temp_file_path}", flush=True)
            s3_client.download_file(S3_bucket_name, s3_file, temp_file_path)
            print(f"Archivo descargado correctamente: {temp_file_path}", flush=True)

            # Subir archivo a GCS
            GCS_file_name = temp_file_name
            print(f"Subiendo: {temp_file_path} -> gs://{GCS_bucket_name}/{GCS_file_name}", flush=True)
            blob = bucket.blob(GCS_file_name)
            blob.upload_from_filename(temp_file_path)
            print(f"Archivo subido correctamente: gs://{GCS_bucket_name}/{GCS_file_name}", flush=True)

            transfer_log.append({
                'archivo_s3': f"s3://{S3_bucket_name}/{s3_file}",
                'archivo_GCS': f"gs://{GCS_bucket_name}/{GCS_file_name}",
                'estado': 'Transferido con éxito'
            })
        except Exception as e:
            print(f"Error al transferir '{s3_file}': {e}", flush=True)
            transfer_log.append({
                'archivo_s3': f"s3://{S3_bucket_name}/{s3_file}",
                'archivo_GCS': None,
                'estado': f"Error: {e}"
            })
        finally:
            # Eliminar archivo temporal
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)
                print(f"Eliminado archivo temporal: {temp_file_path}\n", flush=True)

    # Resumen final
    end_time = datetime.now()
    print(f"=== Proceso de transferencia finalizado | {end_time} ===", flush=True)
    print(f"Duración total: {end_time - start_time}", flush=True)
    print("\n=== Informe de transferencia ===", flush=True)
    for log in transfer_log:
        print(f"- {log['archivo_s3']} -> {log['archivo_GCS']} | Estado: {log['estado']}", flush=True)


In [ ]:
# @title GCS_load_CSV_to_GBQ
import io
import re
import os
import datetime
import unicodedata

import chardet
import pandas as pd
import numpy as np
import dateparser
import pandas_gbq

from google.cloud import storage, bigquery

def GCS_load_CSV_to_GBQ(params: dict) -> None:
    """
    Función unificada que:
      1) Carga archivos CSV desde GCS a BigQuery (forzando todas las columnas como STRING)
         según filtros y nombres indicados.
      2) Inferencia de tipos de columnas (bool, int, float, fecha, string) con muestreo.
      3) Crea (o reemplaza) la tabla definitiva con tipos correctos, y elimina la tabla temporal.

    Parámetros esperados en 'params':
      - project_id (str): ID del proyecto en Google Cloud.
      - source_GCS_bucket_name (str): Nombre del bucket de GCS.
      - source_GCS_file_names_list (list[str]): Lista de nombres de archivos CSV.
        Si está vacío, se importan todos los del bucket (aplicando filtros).
      - source_GCS_file_names_filter (dict): Opciones de filtro para los archivos:
          - use_bool (bool): Activar/desactivar filtros.
          - name_include_patterns_list (list[str]): Patrones a incluir por nombre.
          - name_exclude_patterns_list (list[str]): Patrones a excluir por nombre.
          - extension_include_patterns_list (list[str]): Extensiones permitidas.
          - extension_exclude_patterns_list (list[str]): Extensiones excluidas.
          - min_size_kb (int): Tamaño mínimo (KB).
          - max_size_kb (int): Tamaño máximo (KB).
          - modified_after_date (str): Fecha mínima de modificación (YYYY-MM-DD).
          - modified_before_date (str): Fecha máxima de modificación (YYYY-MM-DD).
          - include_subfolders_bool (bool): Incluir subcarpetas.
      - destination_GBQ_dataset_id (str): ID del dataset de BigQuery.
      - reemplazos (dict): Diccionario de reemplazos en el nombre final de la tabla.
        (ejemplo: {"-utf8": ""})

      -- Parámetros de inferencia de tipos:
      - inference_limit (int): número de filas a muestrear para la inferencia (por defecto 1000).
      - inference_threshold (float): % mínimo de filas no nulas que deben cumplir la condición
                                     para considerar ese tipo (por defecto 0.95).

    Comportamiento:
      - Se cargan los CSV con todas sus columnas como STRING a una tabla temporal <tabla>__TMP.
      - Se infieren tipos: BOOL, INT64, FLOAT64, TIMESTAMP, STRING.
        Además, si en el nombre de columna existe "fecha" o "date", se fuerza a TIMESTAMP/DATE.
      - Se crea (o reemplaza) la tabla final <tabla> con CASTs apropiados.
      - Se elimina la tabla temporal <tabla>__TMP.

    """

    #
    # ============= 1) Definición de sub-funciones auxiliares =============
    #

    def normalize_column_name(name: str) -> str:
        """Normaliza un nombre para BigQuery: ASCII, guiones bajos, máx 300 chars."""
        name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
        name = name.replace('ñ', 'n').replace('Ñ', 'N')
        normalized = re.sub(r"[^a-zA-Z0-9_]", "_", name)
        normalized = re.sub(r"_+", "_", normalized).strip("_")
        return normalized[:300]

    def file_passes_filters(blob) -> bool:
        """Determina si un archivo cumple los filtros especificados en 'source_GCS_file_names_filter'."""
        if not filter_file.get("use_bool", False):
            return True

        blob_name = blob.name
        # Verificar patrones de nombre (include / exclude)
        if filter_file.get("name_include_patterns_list") and not any(pat in blob_name for pat in filter_file["name_include_patterns_list"]):
            return False
        if filter_file.get("name_exclude_patterns_list") and any(pat in blob_name for pat in filter_file["name_exclude_patterns_list"]):
            return False

        # Verificar extensiones
        extension = os.path.splitext(blob_name)[1]
        if filter_file.get("extension_include_patterns_list") and extension not in filter_file["extension_include_patterns_list"]:
            return False
        if filter_file.get("extension_exclude_patterns_list") and extension in filter_file["extension_exclude_patterns_list"]:
            return False

        # Verificar tamaño
        size_kb = blob.size / 1024
        if filter_file.get("min_size_kb") and size_kb < filter_file["min_size_kb"]:
            return False
        if filter_file.get("max_size_kb") and size_kb > filter_file["max_size_kb"]:
            return False

        # Verificar fechas de modificación
        modified_date = blob.updated.date() if blob.updated else None
        if modified_date:
            if filter_file.get("modified_after_date"):
                after_date = datetime.datetime.strptime(filter_file["modified_after_date"], "%Y-%m-%d").date()
                if modified_date < after_date:
                    return False
            if filter_file.get("modified_before_date"):
                before_date = datetime.datetime.strptime(filter_file["modified_before_date"], "%Y-%m-%d").date()
                if modified_date > before_date:
                    return False

        return True

    def detect_encoding_and_load_df(blob):
        """Descarga el blob, detecta la codificación y retorna un DataFrame (asumiendo separador ';')."""
        raw_data = blob.download_as_bytes()
        encoding_detected = chardet.detect(raw_data)['encoding']
        if not encoding_detected:
            encoding_detected = 'utf-8'  # fallback

        raw_text = raw_data.decode(encoding_detected, errors="replace")
        df = pd.read_csv(io.StringIO(raw_text), delimiter=";")
        return df

    #
    # ============= 2) Inferencia de tipos con muestreo =============
    #
    # Mismos helpers de tu segundo script, adaptados dentro de la función principal.

    cache_fecha = {}  # Cache para almacenar resultados de parseo.

    def es_fecha(valor):
        val_str = str(valor).strip()
        if not any(ch.isdigit() for ch in val_str):  # si no hay dígitos, difícilmente sea fecha
            return False
        if val_str in cache_fecha:
            return cache_fecha[val_str]
        try:
            parsed = dateparser.parse(val_str, languages=['en', 'es'])
            result = parsed is not None
            cache_fecha[val_str] = result
            return result
        except Exception:
            cache_fecha[val_str] = False
            return False

    def es_booleano(valor):
        if isinstance(valor, bool):
            return True
        if isinstance(valor, (int, float)):
            return valor in [0, 1]
        if isinstance(valor, str):
            v = valor.strip().lower()
            return v in {"true", "false", "yes", "no", "si", "0", "1"}
        return False

    def normalizar_bool(valor):
        if isinstance(valor, bool):
            return valor
        if isinstance(valor, (int, float)):
            return valor == 1
        if isinstance(valor, str):
            v = valor.strip().lower()
            if v in {"true", "yes", "si", "1"}:
                return True
            elif v in {"false", "no", "0"}:
                return False
        return None

    def es_entero(valor):
        try:
            if isinstance(valor, (int, np.integer)):
                return True
            if isinstance(valor, float) and valor.is_integer():
                return True
            int(str(valor))
            return True
        except:
            return False

    def es_flotante(valor):
        try:
            float(valor)
            return True
        except:
            return False

    def inferir_tipo(serie, threshold):
        datos = serie.dropna()
        if datos.empty:
            return "STRING"
        total = len(datos)
        # Verificar booleana
        bool_count = datos.apply(es_booleano).sum()
        if (bool_count / total) >= threshold:
            bool_values = datos[datos.apply(es_booleano)].apply(normalizar_bool)
            if bool_values.nunique() <= 2:
                return "BOOLEAN"
        # Verificar entero
        if (datos.apply(es_entero).sum() / total) >= threshold:
            return "INTEGER"
        # Verificar float
        if (datos.apply(es_flotante).sum() / total) >= threshold:
            return "FLOAT"
        # Verificar fecha
        if (datos.apply(es_fecha).sum() / total) >= threshold:
            return "TIMESTAMP"
        return "STRING"

    # Mapeo final de tipos a BigQuery
    type_mapping = {
        "BOOLEAN": "BOOL",
        "INTEGER": "INT64",
        "FLOAT": "FLOAT64",
        "TIMESTAMP": "TIMESTAMP",
        "STRING": "STRING"
    }

    #
    # ============= 3) Lectura de parámetros y set-up =============
    #

    # Extraer y validar parámetros
    project_id = params.get("project_id")
    bucket_name = params.get("source_GCS_bucket_name")
    file_names = params.get("source_GCS_file_names_list", [])
    dataset_id = params.get("destination_GBQ_dataset_id")
    filter_file = params.get("source_GCS_file_names_filter", {})
    reemplazos = params.get("reemplazos", {})

    inference_limit = params.get("inference_limit", 1000)
    inference_threshold = params.get("inference_threshold", 0.95)

    if not all([project_id, bucket_name, dataset_id]):
        raise ValueError("❌ Faltan parámetros obligatorios: project_id, source_GCS_bucket_name, destination_GBQ_dataset_id.")

    storage_client = storage.Client(project=project_id)
    bq_client = bigquery.Client(project=project_id)

    bucket = storage_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix="", delimiter="/" if not filter_file.get("include_subfolders_bool", False) else None))

    # Si no se especifican nombres, se procesan todos los archivos filtrados
    if not file_names:
        candidate_files = [blob.name for blob in blobs if file_passes_filters(blob)]
    else:
        # De la lista, quedarnos solo con los que pasan filtros
        candidate_files = []
        for blob in blobs:
            if blob.name in file_names and file_passes_filters(blob):
                candidate_files.append(blob.name)

    if not candidate_files:
        print("No se encontraron archivos que cumplan los filtros/nombres especificados.")
        return

    #
    # ============= 4) Proceso principal para cada archivo =============
    #
    for file_name in candidate_files:
        print(f"\n=== Procesando archivo: {file_name} ===")
        blob = bucket.blob(file_name)

        if not blob.exists():
            print(f"❌ El archivo '{file_name}' no existe en el bucket '{bucket_name}'.")
            continue

        # 4.1) Descargar y convertir a DataFrame
        try:
            df = detect_encoding_and_load_df(blob)
            print(f"✔️ Archivo '{file_name}' cargado en DataFrame (columnas: {len(df.columns)})")
        except Exception as e:
            print(f"❌ Error al leer el archivo '{file_name}': {e}")
            continue

        # 4.2) Normalizar nombres de columnas
        original_columns = df.columns.tolist()
        normalized_columns = [normalize_column_name(col) for col in original_columns]
        df.columns = normalized_columns

        # 4.3) Nombre base de la tabla en BigQuery (aplicando reemplazos)
        raw_table_name = os.path.splitext(file_name)[0]
        for old, new in reemplazos.items():
            raw_table_name = raw_table_name.replace(old, new)
        raw_table_name = raw_table_name.replace("-", "_")
        final_table_name = normalize_column_name(raw_table_name)

        # 4.4) Definimos la tabla temporal y la tabla final
        tmp_table_id = f"{project_id}.{dataset_id}.{final_table_name}__TMP"
        final_table_id = f"{project_id}.{dataset_id}.{final_table_name}"

        # 4.5) Subir a BigQuery la tabla temporal, forzando schema de tipo STRING
        #     (Creamos un schema de BigQuery con todas las columnas como STRING,
        #      pero con "description" la columna original, si se quiere.)
        schema = [
            bigquery.SchemaField(col, "STRING", description=f"Original: {original_columns[i]}")
            for i, col in enumerate(normalized_columns)
        ]

        # Guardar DataFrame en CSV (UTF-8) temporal para cargarlo
        local_tmp_file = f"/tmp/{final_table_name}__TMP.csv"
        df.to_csv(local_tmp_file, index=False, sep=";", encoding="utf-8")

        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            skip_leading_rows=1,
            field_delimiter=";",
            quote_character='"',
            allow_quoted_newlines=True,
            encoding="UTF-8",
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            schema=schema,
            max_bad_records=10
        )

        print(f"📤 Cargando datos a la tabla temporal '{tmp_table_id}'...")
        try:
            with open(local_tmp_file, "rb") as file_obj:
                load_job = bq_client.load_table_from_file(file_obj, tmp_table_id, job_config=job_config)
            load_job.result()
            print(f"✔️ Carga finalizada en la tabla temporal: {tmp_table_id}")
        except Exception as e:
            print(f"❌ Error al cargar la tabla temporal '{tmp_table_id}': {e}")
            continue

        # 4.6) Inferir tipos tomando una muestra de la tabla temporal
        #      Extraemos la muestra con un SELECT * LIMIT inference_limit
        print("🔎 Inferencia de tipos en la tabla temporal (muestreo)...")
        sample_query = f"""
            SELECT *
            FROM `{tmp_table_id}`
            LIMIT {inference_limit}
        """
        try:
            sample_df = pandas_gbq.read_gbq(sample_query, project_id=project_id, progress_bar_type=None)
        except Exception as e:
            print(f"❌ Error al leer la muestra para inferencia en '{tmp_table_id}': {e}")
            # Opcionalmente: drop la tabla temporal
            bq_client.delete_table(tmp_table_id, not_found_ok=True)
            continue

        esquema_inferido = {}
        for col in sample_df.columns:
            # Forzamos fecha si el nombre de la columna contiene "fecha" o "date"
            if "fecha" in col.lower() or "date" in col.lower():
                esquema_inferido[col] = "TIMESTAMP"
            else:
                esquema_inferido[col] = inferir_tipo(sample_df[col], inference_threshold)

        # 4.7) Construir el SELECT con los SAFE_CAST
        select_expressions = []
        for col in sample_df.columns:
            tipo_inferido = esquema_inferido[col]
            bq_type = type_mapping.get(tipo_inferido, "STRING")
            # Importante: el col puede tener caracteres que requieran backticks
            expr = f"SAFE_CAST(`{col}` AS {bq_type}) AS `{col}`"
            select_expressions.append(expr)
        select_clause = ",\n  ".join(select_expressions)

        # 4.8) Creamos/Reemplazamos la tabla final con el nuevo esquema
        create_final_sql = f"""
            CREATE OR REPLACE TABLE `{final_table_id}` AS
            SELECT
              {select_clause}
            FROM `{tmp_table_id}`;
        """

        print(f"📤 Creando/Reemplazando la tabla final '{final_table_id}' con tipos inferidos...")
        try:
            bq_client.query(create_final_sql).result()
            print(f"✔️ Tabla final '{final_table_id}' creada con exito.")
        except Exception as e:
            print(f"❌ Error al crear la tabla final '{final_table_id}': {e}")
            # en caso de error, no borramos la tabla temporal por si se quiere inspeccionar
            continue

        # 4.9) Eliminar la tabla temporal
        try:
            bq_client.delete_table(tmp_table_id, not_found_ok=True)
            print(f"🗑️ Tabla temporal '{tmp_table_id}' eliminada.")
        except Exception as e:
            print(f"⚠️ Aviso: no se pudo eliminar la tabla temporal '{tmp_table_id}': {e}")

    print("\n=== Proceso completo finalizado ===")


# EJECUCIONES

In [ ]:
# @title LISTADO DE ARCHIVOS S3
S3_bucket_name = "animum-datalake-landing-euwest3-637423635409" # @param {"type":"string"}
S3_folder_path = "velneo/" # @param {"type":"string"}
params = {
    'S3_bucket_name': S3_bucket_name,
    'S3_folder_path': S3_folder_path
}

# Ejecutar la función
result = S3_folder_and_files_list(params)

import pprint

# Mostrar resultado
pprint.pprint(result)

{'files': ['velneo/ESC-ANOS-utf8.csv',
           'velneo/ESC-ANOS.csv',
           'velneo/ESC-APET-utf8.csv',
           'velneo/ESC-APET.csv',
           'velneo/ESC-AULA-utf8.csv',
           'velneo/ESC-AULA.csv',
           'velneo/ESC-C&M-utf8.csv',
           'velneo/ESC-C&M.csv',
           'velneo/ESC-C&P-utf8.csv',
           'velneo/ESC-C&P.csv',
           'velneo/ESC-CALE-utf8.csv',
           'velneo/ESC-CALE.csv',
           'velneo/ESC-COBR-utf8.csv',
           'velneo/ESC-COBR.csv',
           'velneo/ESC-CONT-utf8.csv',
           'velneo/ESC-CONT.csv',
           'velneo/ESC-CONV-utf8.csv',
           'velneo/ESC-CONV.csv',
           'velneo/ESC-CURS-utf8.csv',
           'velneo/ESC-CURS.csv',
           'velneo/ESC-DIAS-utf8.csv',
           'velneo/ESC-DIAS.csv',
           'velneo/ESC-DISC-utf8.csv',
           'velneo/ESC-DISC.csv',
           'velneo/ESC-EVAC-utf8.csv',
           'velneo/ESC-EVAC.csv',
           'velneo/ESC-EVAL-utf8.csv',
           'veln

In [ ]:
# @title COPIADO DE ARCHIVOS S3 A GCS

S3_bucket_name = "animum-datalake-landing-euwest3-637423635409"  # @param {"type":"string"}
S3_folder_path = "velneo/"  # @param {"type":"string"}
GCS_bucket_name = "vl_00csv_01"  # @param {"type":"string"}

use_bool = True  # @param {"type": "boolean"}
name_include_patterns_list = ["utf8"]  # @param {"type": "raw"}
name_exclude_patterns_list = ["backup"]  # @param {"type": "raw"}
extension_include_patterns_list = [".txt", ".csv"]  # @param {"type": "raw"}
extension_exclude_patterns_list = [".log"]  # @param {"type": "raw"}
min_size_kb = None  # @param {"type": "number"}
max_size_kb = None  # @param {"type": "number"}
modified_before_date = ""  # @param {"type": "date"}
modified_after_date = ""  # @param {"type":"date"}
include_subfolders_bool = True  # @param {"type": "boolean"}

filter_file = {
    "use_bool": use_bool,
    "name_include_patterns_list": name_include_patterns_list,
    "name_exclude_patterns_list": name_exclude_patterns_list,
    "extension_include_patterns_list": extension_include_patterns_list,
    "extension_exclude_patterns_list": extension_exclude_patterns_list,
    "min_size_kb": min_size_kb,
    "max_size_kb": max_size_kb,
    "modified_after_date": modified_after_date,
    "modified_before_date": modified_before_date,
    "include_subfolders_bool": include_subfolders_bool,
}

config = {
    "S3_bucket_name": S3_bucket_name,
    "S3_folder_path": S3_folder_path,
    "GCS_bucket_name": GCS_bucket_name,
    "filter_file": filter_file,
}

# Ejecutar
S3_to_GCS_transfer_file(config)


=== Iniciando proceso de transferencia de archivos | 2025-02-10 08:32:18.669066 ===

Autenticando cliente de s3...
Autenticando cliente de Google Cloud Storage...
Listando archivos en s3 bucket: 'animum-datalake-landing-euwest3-637423635409', carpeta: 'velneo/'

Aplicando filtros definidos en 'filter_file':
{'use_bool': True, 'name_include_patterns_list': ['utf8'], 'name_exclude_patterns_list': ['backup'], 'extension_include_patterns_list': ['.txt', '.csv'], 'extension_exclude_patterns_list': ['.log'], 'min_size_kb': None, 'max_size_kb': None, 'modified_after_date': '', 'modified_before_date': '', 'include_subfolders_bool': True}
Filtrando por include_patterns=['utf8']: 103 -> 51
Filtrando por exclude_patterns=['backup']: 51 -> 51
Filtrando extensiones permitidas=['.txt', '.csv']: 51 -> 51
Excluyendo extensiones=['.log']: 51 -> 51
=== Comenzando a transferir archivos ===

Iniciando transferencia de 'velneo/ESC-ANOS-utf8.csv'
Descargando: s3://animum-datalake-landing-euwest3-6374236354

In [ ]:
# @title CARGA DE GCS CSV A GBQ

params = {
    "project_id": "animum-dev-datawarehouse",
    "source_GCS_bucket_name": "vl_00csv_01",
    "source_GCS_file_names_list": [],  # Si se deja vacío, se tomarán todos los CSV filtrados
    "source_GCS_file_names_filter": {
        "use_bool": True,
        "name_include_patterns_list": ["utf8"],
        "name_exclude_patterns_list": [],
        "extension_include_patterns_list": [".csv"],
        "extension_exclude_patterns_list": [],
        "min_size_kb": 10,
        "max_size_kb": 5000000,
        "modified_after_date": "2023-01-01",
        "modified_before_date": "2050-12-31",
        "include_subfolders_bool": False
    },
    "destination_GBQ_dataset_id": "vl_01raw_01",
    "reemplazos": {
        "-utf8": ""
    },
    "inference_limit": 1000,       # Número de filas muestreadas para inferencia
    "inference_threshold": 0.95    # Umbral mínimo (0.95 = 95%) para asignar tipo de dato
}

# Ejemplo de llamada:
if __name__ == "__main__":
    GCS_load_CSV_to_GBQ(params)



=== Procesando archivo: ESC-APET-utf8.csv ===
✔️ Archivo 'ESC-APET-utf8.csv' cargado en DataFrame (columnas: 24)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_APET__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_APET__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_APET' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_APET' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_APET__TMP' eliminada.

=== Procesando archivo: ESC-C&M-utf8.csv ===
✔️ Archivo 'ESC-C&M-utf8.csv' cargado en DataFrame (columnas: 11)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_C_M__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_C_M__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplaz

<ipython-input-5-605044fd9e7a>:118: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(raw_text), delimiter=";")


✔️ Archivo 'ESC-COBR-utf8.csv' cargado en DataFrame (columnas: 34)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_COBR__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_COBR__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_COBR' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_COBR' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_COBR__TMP' eliminada.

=== Procesando archivo: ESC-CONT-utf8.csv ===


<ipython-input-5-605044fd9e7a>:118: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(raw_text), delimiter=";")


✔️ Archivo 'ESC-CONT-utf8.csv' cargado en DataFrame (columnas: 41)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_CONT__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_CONT__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_CONT' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_CONT' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_CONT__TMP' eliminada.

=== Procesando archivo: ESC-CONV-utf8.csv ===
✔️ Archivo 'ESC-CONV-utf8.csv' cargado en DataFrame (columnas: 68)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_CONV__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_CONV__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehou

...

✔️ Archivo 'ESC-MATL-utf8.csv' cargado en DataFrame (columnas: 50)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_MATL__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_MATL__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_MATL' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.ESC_MATL' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_MATL__TMP' eliminada.

=== Procesando archivo: ESC-MESE-utf8.csv ===
✔️ Archivo 'ESC-MESE-utf8.csv' cargado en DataFrame (columnas: 10)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.ESC_MESE__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.ESC_MESE__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehou

<ipython-input-5-605044fd9e7a>:118: DtypeWarning: Columns (23,24,25,32,33,35,38,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(raw_text), delimiter=";")


✔️ Archivo 'MA-DIRECCIONES-utf8.csv' cargado en DataFrame (columnas: 45)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_DIRECCIONES__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.MA_DIRECCIONES__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.MA_DIRECCIONES' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.MA_DIRECCIONES' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_DIRECCIONES__TMP' eliminada.

=== Procesando archivo: MA-ENINC-utf8.csv ===
✔️ Archivo 'MA-ENINC-utf8.csv' cargado en DataFrame (columnas: 7)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_ENINC__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.MA_ENINC__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la 

<ipython-input-5-605044fd9e7a>:118: DtypeWarning: Columns (6,7,33,35,53,65,66,69,72,73,77,79,85,102,117,118,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(raw_text), delimiter=";")


✔️ Archivo 'MA-ENTIDADES-utf8.csv' cargado en DataFrame (columnas: 124)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_ENTIDADES__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.MA_ENTIDADES__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla final 'animum-dev-datawarehouse.vl_01raw_01.MA_ENTIDADES' con tipos inferidos...
✔️ Tabla final 'animum-dev-datawarehouse.vl_01raw_01.MA_ENTIDADES' creada con exito.
🗑️ Tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_ENTIDADES__TMP' eliminada.

=== Procesando archivo: MA-PAISES-utf8.csv ===
✔️ Archivo 'MA-PAISES-utf8.csv' cargado en DataFrame (columnas: 14)
📤 Cargando datos a la tabla temporal 'animum-dev-datawarehouse.vl_01raw_01.MA_PAISES__TMP'...
✔️ Carga finalizada en la tabla temporal: animum-dev-datawarehouse.vl_01raw_01.MA_PAISES__TMP
🔎 Inferencia de tipos en la tabla temporal (muestreo)...
📤 Creando/Reemplazando la tabla 